Data Source: https://designcensus.org/

# Table of Content
## 1. [Data Preprocessing](#1.0)
## 2. [Barplots](#2.0)
- ### [Distribution Table for each column](#2.1)
- ### [All/AIGA Count Plot for each column](#2.2)
- ### [Salary vs. Column plots](#2.3)

## 3. [Heatmaps](#3.0)

<br>
<br>
<br>
# Import libraries

In [2]:
import numpy as np
import pandas as pd
import math
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
import locale
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
locale.setlocale(locale.LC_ALL, '')
%matplotlib inline
import pickle

<a id='1.0'></a>
# Part I, Pre-processing

In [ ]:
df = pd.read_csv('DesignCensus2017_Data.csv', encoding = 'utf8')
print('There a a total of ' + str(len(df.columns)) + ' columns.')

### Column name dictionary

In [ ]:
col_dict = {}
for col in df.columns:
    index, desc = col.split(' --')
    col_dict['V_' + str(index)] = desc

In [ ]:
def df_with_column_names(df, col_dict):
    df_v2 = df.copy()
    df_v2.columns = [col_dict[col] for col in df_v2.columns]
    return df_v2

In [ ]:
col_dict

### Classify columns (continuous, categorical, multiple pipe)

In [ ]:
df.columns = map(lambda x : 'V_' + str(x+1), list(range(len(df.columns)))) 
df['V_36'] = df['V_36'].astype(np.object) # ZIPCODE

In [ ]:
continous_cols = ['V_18', 'V_37']
pipe_col = ['V_2', 'V_3','V_5','V_7','V_11','V_19',\
            'V_24', 'V_27', 'V_28', 'V_31', 'V_32',\
            'V_33', 'V_40', 'V_41', 'V_42', 'V_43']

In [ ]:
cate_cols = list(set(df.columns) - set(continous_cols))
cate_cols.sort(key = lambda x : int(x.split('_')[1]))

In [ ]:
# cate_cols_v2 contains columns without pipes and 3 bad columns
cate_cols_v2 = list(set(cate_cols) - set(pipe_col))
cate_cols_v2.remove('V_35') # "Right now, I can't stop listening to:" - No fixed options
cate_cols_v2.remove('V_36') # "I live in:" - Zipcode
cate_cols_v2.remove('V_8') # "I'm not working because:" - null
cate_cols_v2.sort(key = lambda x : int(x.split('_')[1]))

### Summary of null value for each column

In [ ]:
dff = pd.DataFrame(columns=['Column','Desc', 'Null'])
for col in df.columns:
    num_null = df[col].isnull().sum()
    dff = dff.append({'Column': col, 'Desc' : col_dict[col], 'Null' : num_null }, ignore_index=True)
dff.sort_values(by = "Null", ascending= False).head(10)

### Take a look at Salary Outliers

In [ ]:
print('The 99% percentile of salary is: ' + \
      str(int(np.percentile(df.V_18.dropna(), 99))))

print('The 90% percentile of salary is: ' + \
      str(int(np.percentile(df.V_18.dropna(), 90))))

print('The 85% percentile of salary is: ' + \
      str(int(np.percentile(df.V_18.dropna(), 85))))

In [ ]:
# take a look at salary vs. age
dff = pd.DataFrame()
dff['salary'] = df['V_18'] / 1000
dff['age'] = df['V_37']
dff['salary over million'] = dff['salary'] > 1000
sns.lmplot(data = dff, x = 'salary', y = 'age', fit_reg = False, hue = 'salary over million')
plt.xlabel('Salary (thousand)')
plt.ylabel('Age')

In [ ]:
# export a separate file with salary over million record
# df_with_column_names(df[df.V_18 > 1000000], col_dict).to_csv('output_salary_over_million.csv', index = False)

### Break "Age" into groups

In [ ]:
def age_break(x):
    if (x < 20): return ('Younger than 20')
    elif (x <= 25): return ('20 to 25')
    elif (x <= 30): return ('25 to 30')
    elif (x <= 35): return ('30 to 35')
    elif (x <= 40): return ('35 to 40')
    elif (x <= 45): return ('40 to 45')
    elif (x <= 50): return ('45 to 50')
    else: return ('Older than 50')

df['V_37'] = df['V_37'].apply(lambda x : age_break(x))
cate_cols.append('V_37')
continous_cols.remove('V_37')

### Exclude outliers for salary

In [ ]:
## Remove outliers for salary
# 1. Null values 
# 2. zero salary
# 3. Top 1% salary

df = df[~((df.V_18.isnull()) | 
          (df.V_18 == 0) | 
          (df.V_18 > 210000))]

df = df.reset_index()
df = df.drop('index', axis = 1)

### **Done for Jon to export raw data with and without salary outliers

In [ ]:
'''
raw_df = pd.read_csv('DesignCensus2017_Data.csv')
salary = raw_df['18 --My annual salary is:']
raw_df_salary_outliers  = raw_df [((salary.isnull()) | (salary == 0) | (salary > 210000))]
raw_df_excluding_outliers  = raw_df [~((salary.isnull()) | (salary == 0) | (salary > 210000))]
raw_df_salary_outliers.to_csv('./export_csv/raw_data_salary_outliers.csv', index = False, encoding = 'utf8')
raw_df_excluding_outliers.to_csv('./export_csv/raw_data_excluding_salary_outliers.csv', index = False, encoding = 'utf8')
del raw_df, raw_df_salary_outliers, raw_df_excluding_outliers
'''

### Script to deal with columns with pipe

In [ ]:
df = df.replace(np.nan,'', regex=True)
pipe_col_dict = {}

for col in pipe_col:
    options = set()
    for row in df[col]:
        arrs = row.split('|') 
        for arr in arrs:
            if arr != '':
                options.add(arr)
    
    pipe_col_dict[col] = {}
    
    index = 1
    for option in options:
        sub_col = col + '_' + str(index)
        df.loc[:,sub_col] = 0
        pipe_col_dict[col][option] = sub_col
        col_dict[sub_col] = col_dict[col] + '-' + option
        index += 1

In [ ]:
# this part takes a few minutes to run
for col in pipe_col:
    row_num = 0;
    for row in df[col]:
        arrs = row.split('|') 
        for arr in arrs:
            if arr != '':
                sub_col = pipe_col_dict[col][arr]
                df.loc[row_num, sub_col] += 1
        row_num += 1

### Output csv w/ and w/o column names

In [ ]:
df.to_csv('./export_csv/processed_output_without_column_names.csv', index = False, encoding = 'utf8')
df_with_column_names(df, col_dict).to_csv(\
    './export_csv/processed_output_with_column_names.csv', index = False, encoding = 'utf8')    

### Export some important variables

In [ ]:
with open('SavedVariables.pckl', 'wb') as f:
    pickle.dump([continous_cols, cate_cols, cate_cols_v2, pipe_col, col_dict, pipe_col_dict], f)

<br>
<br>
<br>
<br>
<a id='2.0'></a>
# Part II, Barplots

<a id='2.1'></a>
### Read in processed data

In [6]:
df = pd.read_csv('./export_csv/processed_output_without_column_names.csv', encoding = 'utf8')
with open('SavedVariables.pckl', 'rb') as f:
    continous_cols, cate_cols, cate_cols_v2, pipe_col, col_dict, pipe_col_dict = pickle.load(f)

### Basic Summary of each column (only show `top k`)

In [17]:
def summary_cate_col(col, top_k, AllorAIGA = 'All'): #default is all records
    
    AIGA_col = pipe_col_dict['V_32']['AIGA']
    if AllorAIGA == 'All':
        df_filtered = df
    else:
        df_filtered = df[df[AIGA_col] == 1]
        
    if col in continous_cols:
        return
    
    elif col in pipe_col:
        dff = pd.DataFrame(columns=['Options','Count','Percent'])
        for key, value in pipe_col_dict[col].items():
            count = df_filtered[value].sum()
            dff = dff.append({'Options': key, 'Count' : count}, ignore_index=True)
    else:
        dff = pd.DataFrame(df_filtered[col].value_counts())
        dff.reset_index(inplace=True)
        dff.columns = ['Options','Count']
    
    dff['Percent'] = dff.Count / len(df_filtered)
    dff = dff.sort_values(by = 'Percent', ascending = False).head(top_k)
    return dff


### Distribution (%) for each column

In [18]:
for col in cate_cols:
    if 'V_8' in col:
        continue
    if 'V_35' in col:
        continue
    if 'V_36' in col:
        continue
    display(str(col) + ' : ' + col_dict[col])
    dff = summary_cate_col(col, 1000)
    display_dff = dff.style.format({'Percent': '{:,.2%}'.format})
    display(display_dff)

"V_1 :  I've worked in design for:"

,Options,Count,Percent
0,1-4 years,3954,31.69%
1,5-9 years,3597,28.83%
2,10-14 years,1986,15.92%
3,15-20 years,1234,9.89%
4,20+ years,1148,9.20%
5,Less than 1 year,558,4.47%


'V_2 : I primarily work in:'

,Options,Count,Percent
9,Graphic Design,8537,68.42%
15,Web Design,4709,37.74%
27,UX Design,4097,32.84%
0,UI Design,3801,30.46%
18,Marketing,3793,30.40%
16,Communication Design,3425,27.45%
6,Interaction Design,2476,19.84%
19,Product Design,2356,18.88%
17,Illustration,2348,18.82%
20,Experience Design,2103,16.86%


"V_3 : I'm also good at:"

...

,Options,Count,Percent
1,AI and machine learning,6660,53.38%
2,Augmented Reality,5076,40.68%
11,Virtual Reality,4815,38.59%
14,Behavior tracking and modeling,4334,34.74%
0,Internet of Things,3629,29.09%
19,Distributed teams and virtual workplace,3373,27.03%
7,3D printing,3313,26.55%
15,Democratization of design,3275,26.25%
10,Algorithmic design,3100,24.85%
16,Crowdsourcing and open source,2818,22.59%


'V_43 : The most valuable design skills for the future will be (answer optional):'

,Options,Count,Percent
5,Adaptability to tech and social change,8726,69.94%
16,Communication skills,7286,58.40%
17,Empathy,5927,47.50%
2,Asking good questions,5440,43.60%
10,Cross-functional skills,5239,41.99%
13,Storytelling,5189,41.59%
3,Cross-cultural skills,4926,39.48%
0,Observation and listening,4829,38.70%
9,Psychology and human behavior,4682,37.53%
22,Interpersonal skills,4662,37.36%


'V_37 : My age is:'

,Options,Count,Percent
0,25 to 30,3809,30.53%
1,20 to 25,2625,21.04%
2,30 to 35,2541,20.37%
3,35 to 40,1363,10.92%
4,40 to 45,845,6.77%
5,Older than 50,686,5.50%
6,45 to 50,563,4.51%
7,Younger than 20,45,0.36%


<a id='2.2'></a>
### ** Add plots for each column (All or AIGA-only)

In [19]:
AIGA_col = pipe_col_dict['V_32']['AIGA']
print('The total numeber of AIGA members is: ' + str(len(df[df[AIGA_col] == 1])))

The total numeber of AIGA members is: 4638


In [20]:
def plot_single_column(dff, AllorAIGA):
    # plot bar plot for each option
    plt.figure(figsize=(15, 10))
    sns.set_context("notebook", font_scale=1.5)
    g = sns.barplot(y='Options', x = 'Count',data=dff, color = 'blue', ci = None)
    max_count = max(dff['Count'])
    plt.xlim(0, max_count * 1.5)  

    # add labels
    ax = plt.gca()
    for p,count, percent in zip(ax.patches,dff['Count'],dff['Percent']):
        
        ax.annotate(str('{:,}'.format(count)) +' (' + str('{:,.0%}'.format(percent)) +')', 
                    (p.get_x() + p.get_width()+max_count/50 ,p.get_y()+p.get_height()/2 ),# location
                    xytext=(0, 0), textcoords='offset points',# offset
                    va="center",ha="left", size = 12, rotation=0) #text align and font size
     
    # add info and save
    plt.title(AllorAIGA + '_Distribution of "' + col_dict[col] + '"')
    plt.ylabel(col_dict[col])
    plt.xlabel('Count')
    plt.tight_layout()
    
    plt.savefig('./fig/'+ AllorAIGA + '_Single_Column_' + str(col_dict[col]) + '_Plot.png',dpi=100,format='png')
    plt.savefig('./svgfig/'+ AllorAIGA + '_Single_Column_' + str(col_dict[col]) + '_Plot.svg',dpi=100,format='svg')
    plt.gcf().clear()

In [ ]:
for col in cate_cols:
    if 'V_8' in col:
        continue
    if 'V_35' in col:
        continue
    if 'V_36' in col:
        continue
    
    # get summary table each column (All)
    dff = summary_cate_col(col, 1000, AllorAIGA = 'All')
    plot_single_column(dff, AllorAIGA = 'All')
    
    # get summary table each column (AIGA-Only)
    dff = summary_cate_col(col, 1000, AllorAIGA = 'AIGA-Only')
    plot_single_column(dff, AllorAIGA = 'AIGA-Only')

<a id='2.3'></a>
### Plot `salary` vs. all columns (barplots)

In [23]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [ ]:
# All figs are saved under 'fig' folder
sns.set_context("notebook", font_scale=2)
for col in cate_cols_v2:
    groupedvalues = df.groupby(col, as_index = False).agg({'V_18': [np.mean, percentile(95), percentile(5),'count']})
    groupedvalues.columns = [col, 'V_18_avg', 'V_18_max', 'V_18_min','V_18_count']
    groupedvalues.sort_values(by = 'V_18_avg', inplace=True)
    order = list(groupedvalues[col])
    
    plt.figure(figsize=(15,10))
    colorPalette = sns.cubehelix_palette(len(order))
    g=sns.barplot(x='V_18', y=col, data=df, estimator= np.mean, \
                order = order,  palette=colorPalette, ci = None)
    plt.xlim(0,150000)
    
    # add data labels
    ax = plt.gca()
    for p,vmin,vavg,vmax,count in zip(ax.patches,groupedvalues['V_18_min'],groupedvalues['V_18_avg'],
                                                 groupedvalues['V_18_max'],groupedvalues['V_18_count']):
        ax.annotate('5%: ' + "{0:n}".format(int(vmin)) +  # value
                    '\nAvg: ' + "{0:n}".format(int(vavg)) +
                    '\n95%: ' + "{0:n}".format(int(vmax)) + 
                    '\nCount: ' + "{0:n}".format(int(count)),
                    (p.get_x() + p.get_width(),p.get_y()+0.5*p.get_height()),# location
                    xytext=(5, 0), textcoords='offset points',# offset
                    va="center", size = 12) #text align and font size
        
    #plot axes and export
    plt.xlabel('Salary')
    plt.ylabel(col_dict[col])
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig('./fig/'+ 'Salary vs. ' + col_dict[col] + '_Plot.png',dpi=100)
    plt.savefig('./svgfig/'+ 'Salary vs. ' + col_dict[col] + '_Plot.svg',dpi=100, format='svg')
    plt.gcf().clear()

In [25]:
# 2. For columns with pipes
dff = pd.DataFrame(columns = ['Col', 'Option', 'Salary'])
for col in pipe_col:
    for option, sub_col in pipe_col_dict[col].items():
        salary_avg = df.loc[df[sub_col] == 1, 'V_18'].mean()
        salary_min = np.percentile(df.loc[df[sub_col] == 1, 'V_18'],5)
        salary_max = np.percentile(df.loc[df[sub_col] == 1, 'V_18'],95)
        count = len(df.loc[df[sub_col] == 1, 'V_18'])
        dff = dff.append({'Col': col, 'Option' : option, 'Salary_min' : salary_min,
                          'Salary_avg' : salary_avg, 'Salary_max' : salary_max, 'Count' :count }, \
                         ignore_index=True)

In [ ]:
# All figs are saved under 'fig' folder
sns.set_context("notebook", font_scale=1.5)
for col in pipe_col:
    dfff = dff[dff.Col == col].sort_values(by = 'Salary_avg')
    order = list(dfff['Option'])
    plt.figure(figsize=(15,10))
    colorPalette = sns.cubehelix_palette(len(order), rot=-.75)
    sns.barplot(x = 'Salary_avg', y = 'Option', data = dfff, estimator= np.mean, \
                order = order, palette=colorPalette)
    plt.xlim(0, max(150000, max(dfff['Salary_avg']) + 120000)) 
    
    # add data labels
    ax = plt.gca()
    for p,vmin,vavg,vmax,count in zip(ax.patches,dfff['Salary_min'], dfff['Salary_avg'],
                                                 dfff['Salary_max'], dfff['Count']):
        ax.annotate('5%: ' + "{0:n}".format(int(vmin)) +  # value
                    '; Avg: ' + "{0:n}".format(int(vavg)) +
                    '; 95%: ' + "{0:n}".format(int(vmax)) +
                    '; Count: ' + "{0:n}".format(int(count)),
                    (p.get_x() + p.get_width(),p.get_y()+0.5*p.get_height()),# location
                    xytext=(5, 0), textcoords='offset points',# offset
                    va="center", size = 12) #text align and font size 
    #plot axes and export    
    plt.xlabel('Salary')
    plt.ylabel(col_dict[col])
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig('./fig/'+ 'Salary vs. ' + col_dict[col] + '_Plot.png',dpi=100)
    plt.savefig('./svgfig/'+ 'Salary vs. ' + col_dict[col] + '_Plot.svg',dpi=100, format='svg')
    plt.gcf().clear()

<br>
<br>
<br>
<a id='3.0'></a>
# Part III, Heatmaps

<a id='2.1'></a>
### Read in processed data

In [ ]:
df = pd.read_csv('./export_csv/processed_output_without_column_names.csv', encoding = 'utf8')
with open('SavedVariables.pckl', 'rb') as f:
    continous_cols, cate_vols, cate_cols_v2, pipe_col, col_dict, pipe_col_dict = pickle.load(f)

### Heatmap Analysis

In [ ]:
def plot_heatmap(raw_df, col_1, col_1_desc, col_2, col_2_desc, col_salary):
    df = raw_df.copy()
    sns.set(font_scale = 1)
    df[col_salary] = df[col_salary].apply(lambda x : np.round_(x/1000))
    pivot_table=pd.pivot_table(df, index=col_1, columns=col_2, 
                               values = col_salary, aggfunc=np.mean)
    colorPalette = sns.cubehelix_palette()
    sns.heatmap(pivot_table, annot=True, cmap=colorPalette)
    plt.xlabel(col_2_desc)
    plt.ylabel(col_1_desc)
    plt.title('Average Salary Comparison (K)')
    plt.savefig('./heatmaps/' + col_1_desc + ' vs. ' + col_2_desc + '.svg',format='svg')

In [ ]:
# 1. Gender and Orientation Combo
plot_heatmap(df[df['V_38'] != 'Other'], # exclude GENDER == 'Other'
             'V_38', 'Gender', 
             'V_39', 'Orientation',
             'V_18') #salary

In [ ]:
# 2. Process RACE and combine with GENDER, ORIENTATION (duplicate rows)
hm_heat = pd.DataFrame(columns=['index','Gender', 'Orientation','Race', 'Salary'])# v38, v39, v40, v18
for index, row in df.iterrows():
    for option, subcol in pipe_col_dict['V_40'].items(): #race
        if row[subcol] == 1:
            hm_heat = hm_heat.append({'index': index,
                                      'Gender': row['V_38'],  #gender
                                      'Orientation': row['V_39'], #orientation
                                      'Race': option, 
                                      'Salary': row['V_18']}, ignore_index=True)

In [ ]:
# 2. Race and Gender Combo
plot_heatmap(hm_heat[hm_heat['Gender'] != 'Other'], # exclude GENDER == 'Other'
             'Race', 'Race',
             'Gender', 'Gender', 
             'Salary') #salary

In [ ]:
# 3. Race and Orientation Combo
plot_heatmap(hm_heat[hm_heat['Gender'] != 'Other'], # exclude GENDER == 'Other'
             'Race', 'Race',
             'Orientation', 'Orientation', 
             'Salary') #salary

In [ ]:
# 4. Gender and Education
plot_heatmap(df[df['V_38'] != 'Other'], # exclude GENDER == 'Other'
             'V_38', 'Gender', 
             'V_26', 'Higest Education',
             'V_18') #salary

In [ ]:
# Grouped boxplot
sns.boxplot(x="V_18", y="V_26", hue='V_38', data=df[df['V_38'] != 'Other'], orient="h", palette="Set2")
ax = sns.stripplot(x='V_18', y='V_26', data=df[df['V_38'] != 'Other'], color="black", jitter=0.4, size=1)

# TRY VIOLIN PLOT TO VISUALIZE DISTRIBUTION... NEXT

# How can I include the number of observations that go into each of the bars?
# Just the number of rows that were used for any calculations...

plt.savefig('./boxplot/EducationAndGender_Salary.svg',format='svg')

### (Not used now) Generate one-hot coded columns for categorical variables

In [ ]:
for col in cate_cols_v2:
    df.replace(np.nan,'', regex=True, inplace=True)
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(df[col])
    feature = feature.reshape(df.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    for i in range(feature.shape[1]):
        df[col + '-' + str(i + 1)] = np.array(np.matrix(feature).transpose())[i]
        col_dict[col + '-' + str(i + 1)] = col_dict[col] + '-' + label_encoder.classes_[i]
    df.drop(col, axis = 1, inplace = True)

for col in pipe_col:
    df.drop(col, axis = 1, inplace = True)

for col in ['V_8', 'V_35', 'V_36']:
    df.drop(col, axis = 1, inplace = True)

In [ ]:
# output a csv with all columns converted to numeric
df.to_csv('./export_csv/processed_output_all_numeric.csv', index = False, encoding='utf8')